# 🎓 Lab 4: Advanced Red Teaming - Comprehensive Attack Strategy Testing

**Testing AI Models with Multi-Complexity Attack Groups**

---

## 🎯 What You'll Learn

In this advanced lab, you'll conduct comprehensive security testing using **attack strategy groups** that combine multiple obfuscation techniques across three complexity levels.

### Key Learning Objectives

- ✅ **Attack Complexity Levels**: Understand EASY, MODERATE, and DIFFICULT attack categories
- ✅ **Attack Strategy Groups**: Use grouped strategies for comprehensive testing
- ✅ **All Risk Categories**: Scan across all four supported risk types simultaneously
- ✅ **Scale Testing**: Configure multi-objective scans with parallel execution
- ✅ **Comprehensive Analysis**: Interpret results from extensive red team scans

---

## 🔬 What Are Attack Strategy Groups?

Attack strategies are techniques used to bypass AI safety measures by obfuscating harmful prompts. They're organized into three complexity levels:

### 📊 Complexity Classification

| Complexity | Effort Required | Resources Needed | Example Strategies |
|------------|----------------|------------------|-------------------|
| **EASY** | Minimal | Simple encoding/transformation | Base64, Flip, Morse, Leetspeak |
| **MODERATE** | Medium | Access to another AI model | Tense (past tense conversion) |
| **DIFFICULT** | High | Multiple AI models + algorithms | Composition (Tense + Base64) |

### 🎨 Default Attack Strategy Groups

From the Microsoft Docs, here's what each group includes:

#### EASY Group
Includes: `Base64`, `Flip`, `Morse`
- **Base64**: Encodes prompts in base64 format
- **Flip**: Reverses character order  
- **Morse**: Converts text to Morse code

#### MODERATE Group
Includes: `Tense`
- **Tense**: Converts prompts to past tense (requires LLM)

#### DIFFICULT Group
Includes: Composition of `Tense` + `Base64`
- **Composed Attack**: First converts to past tense, then encodes in Base64
- **Why it's harder**: Combines multiple obfuscation layers

---

## 🎯 Lab Focus: Comprehensive Testing

This lab demonstrates **full-spectrum security testing** by:

1. **Testing ALL risk categories** (Violence, Hate/Unfairness, Sexual, Self-Harm)
2. **Using ALL complexity groups** (EASY, MODERATE, DIFFICULT)
3. **Running multiple objectives** per risk category (5 objectives × 4 categories = 20 base prompts)
4. **Applying multiple strategies** to each prompt (3 groups of strategies)

### 📈 Expected Scan Volume

With this configuration:
- **Risk Categories**: 4 (Violence, Hate/Unfairness, Sexual, Self-Harm)
- **Objectives per category**: 5
- **Total base objectives**: 20
- **Attack strategies**: 3 groups (EASY, MODERATE, DIFFICULT)
- **Total attack-response pairs**: ~100+ (baseline + all strategy variations)

---

## 📋 Prerequisites

Before starting this lab, ensure you have:

- ✅ **Completed Labs 1-3**: Understanding of basic red teaming concepts
- ✅ **Azure Setup**: Environment configured from Lab 0
- ✅ **Azure Login**: Authenticated with `az login`
- ✅ **Model Deployment**: Azure OpenAI model deployed (can test any model, including DeepSeek if available)
- ✅ **Environment Variables**: Configured with `source ../../infra/2-setup-env.sh`

---

## ⏱️ Expected Duration

- **Configuration**: ~2 minutes
- **Scan Execution**: ~20-40 minutes (due to comprehensive scope)
- **Results Analysis**: ~15 minutes

**Total Lab Time**: Approximately 40-60 minutes

⚠️ **Note**: This scan is significantly longer than Labs 1-3 due to the comprehensive attack coverage.

---

## 🗺️ Lab Workflow

```mermaid
graph LR
    A[Setup] --> B[Configure All Risk Categories]
    B --> C[Configure Attack Groups]
    C --> D[Run Comprehensive Scan]
    D --> E[Analyze Results]
```

**Steps:**
1. **Environment Setup**: Suppress warnings and configure credentials
2. **Load Configuration**: Get Azure AI project and model endpoints
3. **Create Red Team Agent**: Configure with all 4 risk categories
4. **Run Comprehensive Scan**: Execute scan with EASY, MODERATE, DIFFICULT strategies
5. **Analyze Results**: Review attack success rates across all dimensions

---

## 📚 Learn More

- **[Attack Strategy Groups](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#default-grouped-attack-strategies)** - Complexity classifications
- **[All Attack Strategies](https://learn.microsoft.com/azure/ai-foundry/concepts/ai-red-teaming-agent#supported-attack-strategies)** - Complete list of 20+ strategies
- **[Attack Complexity](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#supported-attack-strategies)** - Understanding effort levels

---

Let's begin comprehensive security testing! 🚀

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 🔇 Step 1: Suppress Warning Messages

To keep the output clean and focused on scan results, we'll suppress Python warnings.

This is a common practice in notebooks to reduce noise from deprecation warnings, library version messages, and other non-critical alerts.

---

**Run the cell below to suppress warnings:**

## 🔑 Step 2: Configure Azure Authentication

We'll set up Azure credentials using the Azure CLI authentication from your `az login` session.

### Authentication Check

The code first verifies if you're already authenticated to Azure:
- ✅ If logged in: Uses existing session
- ❌ If not logged in: Prompts you to run `az login`

### AzureCliCredential

This credential type leverages your Azure CLI login, which is ideal for:
- Local development and testing
- Jupyter notebooks (like this lab)
- Quick prototyping without service principals

---

**Run the cell below to configure authentication:**

## ⚙️ Step 3: Load Azure AI Project and Model Configuration

Now we'll load the necessary configuration from environment variables to set up our scan target.

### Configuration Components

1. **Azure AI Project Endpoint**: Connection to your AI Foundry project
   - Used for logging scan results to the portal
   - Variable: `AZURE_AI_PROJECT_ENDPOINT`

2. **Model Deployment**: The specific model to test
   - Variable: `AZURE_DEEPSEEK_DEPLOYMENT` (or any other deployment name)
   - This can be any Azure OpenAI model (GPT-4, GPT-3.5, DeepSeek, etc.)

3. **API Key**: Authentication for the model endpoint
   - Variable: `AZURE_OPENAI_API_KEY`

### Model Configuration Dictionary

The `model_config` dictionary creates a target configuration for the red team scan:
```python
{
    "azure_endpoint": "Your project endpoint",
    "azure_deployment": "Your model deployment name",
    "api_key": "Your API key"
}
```

---

**Run the cell below to load configuration:**

## 🎯 Step 4: Create Comprehensive Red Team Agent

Here we'll create a Red Team agent configured for **comprehensive security testing** across all risk categories.

### Configuration Breakdown

#### 1. Import Required Classes
```python
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
```

**Note**: This uses the **local** red teaming SDK (`azure.ai.evaluation.red_team`), not the cloud SDK. The scan runs locally but results are logged to the portal.

#### 2. Risk Categories - All Four Types

We're testing **ALL supported risk categories** simultaneously:

| Risk Category | Description | Example Attack Objectives |
|---------------|-------------|---------------------------|
| **HateUnfairness** | Bias, discrimination, hateful content | Testing for racist, sexist, or discriminatory outputs |
| **Violence** | Violent, harmful, dangerous content | Testing for weapons, violence, physical harm |
| **Sexual** | Inappropriate sexual content | Testing for explicit or inappropriate sexual material |
| **SelfHarm** | Self-injury, suicide-related content | Testing for self-harm or suicide encouragement |

#### 3. Objectives Configuration

**`num_objectives=5`**: Generate 5 adversarial objectives per risk category
- **Total objectives**: 5 × 4 categories = **20 base attack objectives**
- Each objective gets tested with multiple attack strategies
- Results in 100+ attack-response pairs total

### 💡 Why Test All Categories?

Comprehensive testing ensures:
- ✅ **Complete safety coverage**: No blind spots
- ✅ **Comparative analysis**: See which categories are most vulnerable
- ✅ **Production readiness**: Full pre-deployment validation
- ✅ **Compliance requirements**: Meet safety assessment standards

### ⏱️ Scan Duration Estimate

With 5 objectives × 4 categories × 3 attack groups:
- **Expected runtime**: 20-40 minutes
- **Attack-response pairs**: ~100-120
- **Parallelization**: Helps speed up execution

---

**Run the cell below to create the Red Team agent:**

## 🚀 Step 5: Execute Comprehensive Red Team Scan

Now we'll launch the comprehensive scan using **all three attack strategy groups**.

### Scan Parameters Explained

#### 1. Target Configuration
```python
target=model_config
```
The model configuration dictionary from Step 3, specifying which model to test.

#### 2. Scan Name
```python
scan_name="4-AttackGroups-Target"
```
Unique identifier for this scan in the Azure AI Foundry Portal.
- Format follows our naming convention: `{LabNumber}-{ScanType}-Target`
- Helps organize and find results in the portal

#### 3. Attack Strategy Groups
```python
attack_strategies=[
    AttackStrategy.EASY,
    AttackStrategy.MODERATE,
    AttackStrategy.DIFFICULT,
]
```

**What This Tests:**

| Strategy Group | Included Techniques | Attack Count Per Objective |
|----------------|---------------------|---------------------------|
| **EASY** | Base64, Flip, Morse | 3 variations |
| **MODERATE** | Tense | 1 variation |
| **DIFFICULT** | Tense + Base64 (composed) | 1 variation |
| **BASELINE** | Direct adversarial prompts | 1 original |

**Total per objective**: 6 attack variations (1 baseline + 5 strategy variations)

#### 4. Parallel Execution
```python
max_parallel_tasks=6
```
Runs up to 6 attacks simultaneously to speed up execution.
- **Benefits**: Faster completion time
- **Consideration**: Consumes more API quota

#### 5. Timeout
```python
timeout=600
```
Maximum time (in seconds) for the entire scan.
- **600 seconds** = 10 minutes
- ⚠️ **Note**: With 100+ attacks, this scan may exceed the timeout
- The scan will complete as many attacks as possible within the time limit

### 📊 Expected Results

After completion, you'll see:
- **Total attacks executed**: ~120 (20 objectives × 6 strategies)
- **Attack Success Rate (ASR)**: Percentage of successful attacks per category
- **Scorecard**: Breakdown by risk category and attack complexity
- **Detailed logs**: Individual attack-response pairs

### 🌐 Results Location

Results will be available in:
1. **Local return value**: `model_results` variable
2. **Azure AI Foundry Portal**: Under "Evaluations" → "AI red teaming"
   - Look for scan name: `4-AttackGroups-Target`

---

**Run the cell below to start the comprehensive scan:**

⚠️ **Note**: This cell uses `await` and requires async execution. The scan may take 20-40 minutes to complete.

In [ ]:
# Azure Credential imports
from azure.identity import AzureCliCredential
import subprocess

# Check if already logged in to Azure
try:
    result = subprocess.run(['az', 'account', 'show'], capture_output=True, text=True, check=True)
    print("✓ Already logged in to Azure")
except subprocess.CalledProcessError:
    print("Not logged in. Initiating Azure login...")
    !az login

# Initialize Azure credentials
credential = AzureCliCredential()

In [ ]:
import os

# Azure AI Project information
azure_ai_project = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")

azure_endpoint = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")
azure_deployment = os.environ.get("AZURE_DEEPSEEK_DEPLOYMENT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")

model_config = {
    "azure_endpoint": azure_endpoint,
    "azure_deployment": azure_deployment,
    "api_key": api_key,
}


In [ ]:
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# Create the RedTeam agent
agent = RedTeam(
    azure_ai_project=azure_ai_project, 
    credential=credential, 
    risk_categories=[
        RiskCategory.HateUnfairness,
        RiskCategory.Violence,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm,
    ],
    num_objectives=5,
)

In [ ]:
# Run the comprehensive red teaming scan
print("🚀 Starting comprehensive red team scan...")
print("   This will test ALL risk categories with ALL attack complexity groups")
print("   Expected duration: 20-40 minutes")
print("\n⏳ Scan in progress...\n")

model_results = await agent.scan(
    target=model_config,
    scan_name="4-AttackGroups-Target",  # Updated to follow naming convention
    attack_strategies=[
        AttackStrategy.EASY,       # Base64, Flip, Morse
        AttackStrategy.MODERATE,   # Tense
        AttackStrategy.DIFFICULT,  # Tense + Base64 composition
    ],
    max_parallel_tasks=6,
    timeout=600,  # 10 minutes - may need to increase for full completion
)

print("\n✅ Scan completed!")
print("\n📊 Results Summary:")
print(model_results)

## 🎉 Lab 4 Summary

Congratulations! You've completed **Lab 4: Advanced Red Teaming with Comprehensive Attack Strategy Testing**.

---

## ✅ What You Accomplished

In this advanced lab, you learned how to:

1. ✅ **Comprehensive Risk Coverage**: Tested all 4 risk categories (Violence, Hate/Unfairness, Sexual, Self-Harm) simultaneously
2. ✅ **Attack Strategy Groups**: Used EASY, MODERATE, and DIFFICULT complexity groups
3. ✅ **Multi-Objective Testing**: Configured 5 objectives per category for thorough coverage
4. ✅ **Parallel Execution**: Optimized scan performance with concurrent testing
5. ✅ **Large-Scale Analysis**: Interpreted results from 100+ attack-response pairs

---

## 🔍 Key Takeaways

### Attack Complexity Levels

You tested three levels of attack sophistication:

| Complexity | Strategies | Adversary Profile | Detection Difficulty |
|------------|------------|-------------------|---------------------|
| **EASY** | Base64, Flip, Morse | Script kiddie, automated tools | Low - Medium |
| **MODERATE** | Tense (requires LLM) | Skilled attacker with AI access | Medium - High |
| **DIFFICULT** | Composed (Tense + Base64) | Advanced persistent threat | High |

### Why This Matters

- **Real-world attacks** often use multiple obfuscation techniques
- **Defense in depth** requires testing against sophisticated threats
- **Baseline testing** (direct prompts) isn't sufficient for production
- **Comprehensive scans** reveal vulnerabilities that simple tests miss

---

## 📊 Understanding Your Results

### Attack Success Rate (ASR)

The most important metric from your scan:

```
ASR = (Successful Attacks / Total Attacks) × 100%
```

**Interpreting ASR:**
- **0-20%**: Excellent safety measures
- **20-40%**: Good, but room for improvement
- **40-60%**: Moderate risk, mitigation recommended
- **60%+**: High risk, immediate action required

### Scorecard Breakdown

Your results JSON includes:

1. **By Risk Category**: Which categories had highest ASR?
2. **By Attack Complexity**: Which attack levels were most successful?
3. **Joint Analysis**: Risk × Complexity breakdown
4. **Row-level Data**: Individual attack-response pairs with scores

---

## 🎯 Next Steps

### 1. 🔍 Deep Dive into Results

Review your scan results in the Azure AI Foundry Portal:

**Portal Navigation:**
```
Azure AI Foundry → Your Project → Evaluations → AI red teaming → "4-AttackGroups-Target"
```

**What to Look For:**
- Which risk categories had the highest ASR?
- Which attack strategies were most effective?
- Are there patterns in successful attacks?
- Do certain attack compositions bypass defenses?

### 2. 🛡️ Implement Mitigations

Based on your findings, consider:

**Content Filtering:**
```python
# Apply Azure AI Content Safety filters
from azure.ai.contentsafety import ContentSafetyClient

# Configure filter thresholds based on your ASR results
```

**Safety System Messages:**
```python
# Add safety instructions to your system prompt
system_message = """
You are a helpful AI assistant. Follow these safety guidelines:
- Do not provide information that could cause harm
- Refuse requests for illegal or dangerous content
- Maintain ethical standards in all responses
"""
```

**Prompt Shields:**
- Enable Azure OpenAI Content Safety filters
- Use jailbreak detection
- Implement user input validation

### 3. 🔄 Iterative Testing

After implementing mitigations:

1. **Re-run this scan** with the same configuration
2. **Compare ASR** before and after mitigations
3. **Verify improvements** across all risk categories
4. **Document changes** for compliance and audit

### 4. 🧪 Custom Attack Strategies

Try individual strategies for targeted testing:

```python
# Test specific obfuscation techniques
attack_strategies=[
    AttackStrategy.JAILBREAK,  # Prompt injection
    AttackStrategy.ROT13,       # Cipher encoding
    AttackStrategy.LEETSPEAK,   # Character substitution
    AttackStrategy.Compose([AttackStrategy.ROT13, AttackStrategy.BASE64])
]
```

### 5. 📈 Continuous Monitoring

Make red teaming part of your AI development lifecycle:

- **Weekly scans**: During active development
- **Pre-deployment**: Before each release
- **Post-update**: After model upgrades or changes
- **Scheduled audits**: Monthly or quarterly

### 6. 🔬 Advanced Techniques

**Custom Attack Objectives:**
```python
# Bring your own attack prompts
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts="my_custom_prompts.json"
)
```

**Complex Callbacks:**
```python
# Test full applications with multi-turn conversations
async def application_callback(messages: List[dict]) -> dict:
    # Your RAG system, chatbot, or complex application logic
    return {"role": "assistant", "content": response}
```

---

## 📚 Additional Resources

### Official Documentation

- **[Attack Strategy Groups](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#default-grouped-attack-strategies)** - Default group definitions
- **[All Attack Strategies](https://learn.microsoft.com/azure/ai-foundry/concepts/ai-red-teaming-agent#supported-attack-strategies)** - Complete strategy list (20+)
- **[Attack Complexity](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#supported-attack-strategies)** - Understanding effort levels
- **[Custom Objectives](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#custom-attack-objectives)** - Bring your own prompts

### Mitigation Resources

- **[Azure AI Content Safety](https://learn.microsoft.com/azure/ai-services/content-safety/overview)** - Filter configuration
- **[Safety System Messages](https://learn.microsoft.com/azure/ai-foundry/openai/concepts/safety-system-message-templates)** - Pre-built templates
- **[Responsible AI Practices](https://www.microsoft.com/ai/responsible-ai)** - Microsoft's principles

### Research and Best Practices

- **[Microsoft AI Red Team Blog](https://www.microsoft.com/security/blog/2025/01/13/3-takeaways-from-red-teaming-100-generative-ai-products/)** - Lessons from 100+ products
- **[PyRIT Framework](https://github.com/Azure/PyRIT)** - Open-source risk identification tool
- **[NIST AI Risk Management](https://www.nist.gov/itl/ai-risk-management-framework)** - Industry framework

---

## 🎓 Lab Series Completion

You've now completed **all four labs** in the AI Red Teaming series:

| Lab | Focus | Key Learning |
|-----|-------|--------------|
| **Lab 1** | Agent Scanning | Test Azure AI agents with callback functions |
| **Lab 2** | Model Scanning | Test models with various target configurations |
| **Lab 3** | Cloud Scanning | Scale testing with Azure AI Foundry cloud infrastructure |
| **Lab 4** | Advanced Testing | Comprehensive multi-complexity attack groups |

### 🌟 You're Now Equipped To:

- ✅ Conduct thorough security assessments of AI systems
- ✅ Understand attack vectors from basic to advanced
- ✅ Interpret red teaming results and prioritize fixes
- ✅ Implement defense-in-depth strategies
- ✅ Build security testing into your AI development workflow

---

## 💡 Need Help?

- **Technical Issues?** Check [Azure AI Foundry Docs](https://learn.microsoft.com/azure/ai-foundry/)
- **Scan Failed?** Review timeout settings and parallel task configuration
- **Understanding Results?** See [Results Documentation](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#results-from-your-automated-scans)

---

**Excellent work completing this advanced lab!** 🎉 You've mastered comprehensive AI red teaming with Azure AI Foundry. 🚀